<!-- NOTEBOOK_METADATA source: "⚠️ Jupyter Notebook" title: "TrueFoundry AI Gateway Integration" sidebarTitle: "Truefoundry" logo: "/images/integrations/truefoundry-logo.png" description: "Learn how to integrate Truefoundry with Langfuse using the OpenAI drop-in replacement." category: "Integrations" -->

# TrueFoundry AI Gateway Integration

> **What is Truefoundry?** [TrueFoundry](https://www.truefoundry.com/ai-gateway) is an enterprise-grade AI Gateway and control plane that lets you deploy, govern, and monitor any LLM or Gen-AI workload behind a single OpenAI-compatible API—bringing rate-limiting, cost controls, observability, and on-prem support to production AI applications.

## How Truefoundry Integrates with Langfuse

Truefoundry’s AI Gateway and Langfuse combine to give you enterprise-grade observability, governance, and cost control over every LLM request—set up in minutes.

<details>
<summary><strong>Unified OpenAI-Compatible Endpoint</strong></summary>

Point the Langfuse OpenAI client at Truefoundry’s gateway URL. Truefoundry routes to any supported model (OpenAI, Anthropic, self-hosted, etc.), while Langfuse transparently captures each call—no code changes required.
</details>

<details>
<summary><strong>End-to-End Tracing & Metrics</strong></summary>

Langfuse delivers:
- **Full request/response logs** (including system messages)  
- **Token usage** (prompt, completion, total)  
- **Latency breakdowns** per call  
- **Cost analytics** by model and environment  
Drill into any trace in seconds to optimize performance or debug regressions.
</details>

<details>
<summary><strong>Production-Ready Controls</strong></summary>

Truefoundry augments your LLM stack with:
- **Rate limiting & quotas** per team or user  
- **Budget alerts & spend caps** to prevent overruns  
- **Scoped API keys** with RBAC for dev, staging, prod  
- **On-prem/VPC deployment** for full data sovereignty  
</details>

## Prerequisites

Before integrating Langfuse with TrueFoundry, ensure you have:
1. TrueFoundry Account: Create a [Truefoundry account](https://www.truefoundry.com/register) with atleast one model provider and generate a Personal Access Token by following the instructions in [quick start](https://docs.truefoundry.com/gateway/quick-start) and [generating tokens](https://docs.truefoundry.com/gateway/authentication)
2. Langfuse Account: Sign up for a free [Langfuse Cloud account](https://cloud.langfuse.com/) or self-host Langfuse

<!-- STEPS_START -->
## Step 1: Install Dependencies

In [ ]:
%pip install openai langfuse

## Step 2: Set Up Environment Variables

Next, set up your Langfuse API keys. You can get these keys by signing up for a free [Langfuse Cloud](https://cloud.langfuse.com/) account or by [self-hosting Langfuse](https://langfuse.com/self-hosting). These environment variables are essential for the Langfuse client to authenticate and send data to your Langfuse project.

In [ ]:
import os

# Langfuse Configuration
os.environ["LANGFUSE_PUBLIC_KEY"] = "pk-lf-..."
os.environ["LANGFUSE_SECRET_KEY"] = "sk-lf-..."
os.environ["LANGFUSE_HOST"] = "https://cloud.langfuse.com"  # 🇪🇺 EU region
# os.environ["LANGFUSE_HOST"] = "https://us.cloud.langfuse.com"  # 🇺🇸 US region

# TrueFoundry Configuration
os.environ["TRUEFOUNDRY_API_KEY"] = "your-truefoundry-token"
os.environ["TRUEFOUNDRY_BASE_URL"] = "https://your-control-plane.truefoundry.cloud/api/llm"

In [ ]:
from langfuse import get_client

# Test Langfuse authentication
get_client().auth_check()

True

## Step 3: Use Langfuse OpenAI Drop-in Replacement

Use Langfuse's OpenAI-compatible client to capture and trace every request routed through the TrueFoundry AI Gateway. Detailed steps for configuring the gateway and generating virtual LLM keys are available in the [TrueFoundry documentation](https://docs.truefoundry.com/gateway/langfuse).

In [ ]:
from langfuse.openai import OpenAI
import os

# Initialize OpenAI client with TrueFoundry Gateway
client = OpenAI(
    api_key=os.environ["TRUEFOUNDRY_API_KEY"],
    base_url=os.environ["TRUEFOUNDRY_BASE_URL"]  # Base URL from unified code snippet
)

## Step 4: Run an Example

In [ ]:
# Make a request through TrueFoundry Gateway with Langfuse tracing
response = client.chat.completions.create(
    model="openai-main/gpt-4o",  # Paste the model ID you copied from TrueFoundry Gateway
    messages=[
        {"role": "system", "content": "You are a helpful AI assistant specialized in explaining AI concepts."},
        {"role": "user", "content": "Why does an AI gateway help enterprises?"},
    ],
    max_tokens=500,
    temperature=0.7
)

print(response.choices[0].message.content)

# Ensure all traces are sent to Langfuse
langfuse = get_client()
langfuse.flush()

## Step 5: See Traces in Langfuse

After running the example, log in to Langfuse to view the detailed traces, including:

- Request parameters
- Response content
- Token usage and latency metrics
- LLM model information through Truefoundry gateway

![Langfuse Trace Example](https://langfuse.com/images/cookbook/integration_troufoundry/truefoundry-trace.png)

> **Note**: All other features of Langfuse will work as expected, including prompt management, evaluations, custom dashboards, and advanced observability features. The TrueFoundry integration seamlessly supports the full Langfuse feature set.
<!-- STEPS_END -->

## Advanced Integration with Langfuse Python SDK

Enhance your observability by combining the automatic tracing with additional Langfuse features.

### Using the @observe Decorator

The `@observe()` decorator automatically wraps your functions and adds custom attributes to traces:

In [ ]:
from langfuse import observe, get_client

langfuse = get_client()

@observe()
def analyze_customer_query(query, customer_id):
    """Analyze customer query using TrueFoundry Gateway with full observability"""
    
    response = client.chat.completions.create(
        model="openai-main/gpt-4o",
        messages=[
            {"role": "system", "content": "You are a customer service AI assistant."},
            {"role": "user", "content": query},
        ],
        temperature=0.3
    )
    
    result = response.choices[0].message.content
    
    # Add custom metadata to the trace
    langfuse.update_current_trace(
        input={"query": query, "customer_id": customer_id},
        output={"response": result},
        user_id=customer_id,
        session_id=f"session_{customer_id}",
        tags=["customer-service", "truefoundry-gateway"],
        metadata={
            "model_used": "openai-main/gpt-4o",
            "gateway": "truefoundry",
            "query_type": "customer_support"
        },
        version="1.0.0"
    )
    
    return result

# Usage
result = analyze_customer_query("How do I reset my password?", "customer_123")

### Debug Mode

Enable debug logging for troubleshooting:

```python
import logging
logging.basicConfig(level=logging.DEBUG)
```

> **Note**: All other features of Langfuse will work as expected, including prompt management, evaluations, custom dashboards, and advanced observability features. The TrueFoundry integration seamlessly supports the full Langfuse feature set.

## Learn More

- **TrueFoundry AI Gateway Introduction**: [https://docs.truefoundry.com/gateway/intro-to-llm-gateway](https://docs.truefoundry.com/gateway/intro-to-llm-gateway)
- **TrueFoundry Authentication Guide**: [https://docs.truefoundry.com/gateway/authentication](https://docs.truefoundry.com/gateway/authentication)



<!-- MARKDOWN_COMPONENT name: "LearnMore" path: "@/components-mdx/integration-learn-more.mdx" -->